In [1]:
! pip install transformers
! pip install datasets

In [2]:
import os
os.chdir('/content/drive/MyDrive/CAMemBERT2')
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification,AutoModel
import numpy as np
from math import floor
import torch
from torch.cuda import is_available
from collections import defaultdict
from scipy.stats import spearmanr
from sklearn.metrics import cohen_kappa_score,classification_report,fbeta_score
from models_f import MultiTaskModel
from trainer import MultiTaskModelTrainer
from trainer_utils import training_args,training_kwargs
from preprocessing_f import CreateHuggingFaceMultiTask


In [3]:
dataset_obj = CreateHuggingFaceMultiTask()
dataset_dict = dataset_obj.get_combined_dataset_dict()

/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{self.set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','labels'])
/content/drive/MyDrive/CAMemBERT2/preprocessing_f.py:53: ParserWarning: Fa

  0%|          | 0/1057 [00:00<?, ?ex/s]

  0%|          | 0/97 [00:00<?, ?ex/s]

  0%|          | 0/79 [00:00<?, ?ex/s]

  0%|          | 0/1057 [00:00<?, ?ex/s]

...

  0%|          | 0/266 [00:00<?, ?ex/s]

  0%|          | 0/243 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
model_kwargs = training_kwargs(
    class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
    shared_encoder_n_layers=3,
    trainer=False)
trainer_kwargs = training_kwargs(
    class_weights={'aes':None,'ged':dataset_obj.get_weights(),'ner':None},
    shared_encoder_n_layers=3,
    trainer=True)
model = MultiTaskModel(kwargs_dict = model_kwargs)
for name,params in model.model.named_parameters():
    if 'classifier' not in name:
        params.requires_grad = False
args = training_args()
trainer = MultiTaskModelTrainer(
    model,
    args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['dev'],
)
trainer.kwargs_ = trainer_kwargs
trainer.test_dataset = dataset_dict['test']
trainer.train()

loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab

1st step


Step,Training Loss
500,37.669000
1000,10.540900
1500,9.738300
2000,9.471000


aes_weight_coef : 1.0
ged_weight_coef : 1.0
ner_weight_coef : 1.0

losses {'aes': 7.676556964253271, 'ged': 0.14643993211347003, 'ner': 0.6149702224620553, 'total_loss': 8.437967118828796}

aes_metrics {'rmse_aes': 6.089608635481704, 'pearson_aes': 0.40873192890690774, 'spearman_aes': 0.4167410784790778, 'kappa_aes': 0.09049528557494313}

ged               precision    recall  f1-score   support

           0       0.87      0.76      0.81     25174
           1       0.13      0.24      0.17      3917

    accuracy                           0.69     29091
   macro avg       0.50      0.50      0.49     29091
weighted avg       0.77      0.69      0.72     29091

ged_metrics {'f1_score_avg_ged': 0.691760338248943, 'f1_score_macro_ged': 0.4910002613282531, 'f1_score_weighted_ged': 0.724583017491532, 'f_0_5_ged': 0.20534290271132377}

ner               precision    recall  f1-score   support

           0  0.832502628 1.000000000 0.908596381     42759
           1  0.000000000 0.00000000

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1st step
aes_weight_coef : 1.0
ged_weight_coef : 1.0
ner_weight_coef : 1.0

losses {'aes': 7.186657816864724, 'ged': 0.14588614951732548, 'ner': 0.6297518999077553, 'total_loss': 7.9622958662898045}

aes_metrics {'rmse_aes': 5.85709446360186, 'pearson_aes': 0.5698867149737882, 'spearman_aes': 0.5562414210584062, 'kappa_aes': 0.3026964077756875}

ged               precision    recall  f1-score   support

           0       0.87      0.68      0.76     25174
           1       0.14      0.33      0.19      3917

    accuracy                           0.63     29091
   macro avg       0.50      0.50      0.48     29091
weighted avg       0.77      0.63      0.68     29091

ged_metrics {'f1_score_avg_ged': 0.6311230277405383, 'f1_score_macro_ged': 0.4777774916276626, 'f1_score_weighted_ged': 0.68455661248223, 'f_0_5_ged': 0.25879256719036603}

ner               precision    recall  f1-score   support

           0  0.832502628 1.000000000 0.908596381     42759
           1  0.000000000 0.0



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2385, training_loss=15.672937753865304, metrics={'train_runtime': 543.7602, 'train_samples_per_second': 116.632, 'train_steps_per_second': 4.384, 'total_flos': 0.0, 'train_loss': 15.672937753865304, 'epoch': 9.0})

In [9]:
trainer.history_dict['train']

defaultdict(list,
            {'epoch': [1.89, 3.77, 5.66, 7.55, 9.43],
             'learning_rate': [0.0004685534591194969,
              0.0004371069182389937,
              0.0004056603773584906,
              0.0003742138364779874,
              0.00034276729559748427],
             'loss': [39.3406, 11.5348, 10.8142, 10.4558, 9.9663],
             'step': [500, 1000, 1500, 2000, 2500]})